# General settings

In [1]:
import sys
import math
from scipy.optimize import minimize, bisect
import numpy as np
from numpy import linalg as nplin

In [2]:
from ekdist import eklib

In [3]:
# LOAD DATA.
filename = "./tests/intervals.txt"
intervals = np.loadtxt(filename)

## Fit multi-exponential pdf

In [4]:
tau, area = [0.036, 1.1], [0.2]
epdf = eklib.ExponentialPDF(tau, area)
print('Start LogLikelihood =', epdf.LL(epdf.theta, intervals))

Start LogLikelihood = 87.31806715582867


In [5]:
res = minimize(epdf.LL, epdf.theta, args=intervals, method='Nelder-Mead')
print (res)

 final_simplex: (array([[0.03700718, 1.07302608, 0.19874548],
       [0.03701042, 1.07300003, 0.19879291],
       [0.03701506, 1.07312038, 0.19880849],
       [0.03700726, 1.07309781, 0.19883809]]), array([87.28828773, 87.28828814, 87.28828846, 87.28828878]))
           fun: 87.28828773327726
       message: 'Optimization terminated successfully.'
          nfev: 88
           nit: 47
        status: 0
       success: True
             x: array([0.03700718, 1.07302608, 0.19874548])


### Calculate approximate SD

In [6]:
from ekdist import errors
asd = errors.ApproximateSD(res.x, epdf.LL, np.array(intervals))

In [7]:
print(asd.hessian)

[[8475.49606292  -92.21425006 -626.06648917]
 [ -92.21425006   71.5478832   -36.17166531]
 [-626.06648917  -36.17166531  501.13966047]]


In [8]:
print(asd.covariance)

[[0.00013478 0.00026864 0.00018777]
 [0.00026864 0.01504143 0.00142128]
 [0.00018777 0.00142128 0.00233261]]


In [9]:
print(asd.sd)

[0.01160948 0.1226435  0.04829715]


In [10]:
print(asd.correlations)

[[1.         0.18867387 0.33487997]
 [0.18867387 1.         0.23994593]
 [0.33487997 0.23994593 1.        ]]


### Calculate likelihood intervals

In [11]:
m = 2.0 # corresponds roughly to 2 SD
#likints = eklib.lik_intervals(res.x, epdf.LL, np.array(intervals), sd, m)